In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

# 금융-비금융 데이터 통합

In [117]:
df0 = pd.read_csv('금융_#4_원문_Cluster2.csv')
df1 = pd.read_csv('비금융_#10_원문_Cluster1.csv')
print(df0.shape, df1.shape)

(16600, 4) (5050, 4)


In [118]:
df = pd.concat([df0, df1], axis =  0, ignore_index=True)

# 데이터 불러오기

In [119]:
#df=pd.read_csv('[KB]1차_Kmeans_원문_Cluster3.csv')

#feather로 확장자명 바꾸면 2배 빨라집니다!
df.to_feather("./KB_Kmeans_cluster5.ftr")
df= pd.read_feather("./KB_Kmeans_cluster5.ftr", columns=None, use_threads=True)
df

,Unnamed: 0,review,Mecab_token,cluster
0,6,일상 IT 화 네이버 쇼핑 특가창고 오픈네이버에서 이번에는 창고 를 만들었습니다 ...,"['가격', '아워홈', '하루', '이야기', '콜라', '구성', '디지털', ...",2
1,7,도매꾹 위탁판매 로 스마트 스토어 도전 늦었어요 한 참 늦었어요 올해 중순이 되어서...,"['후기', '이야기', '하루', '순이익', '잡스', '영상', '대신', '...",2
2,13,부릉부릉 스타트업 일 방송분 다시 듣기 http www kbs co kr radio...,"['인터넷', '공급자', '이야기', '입력', '추천', '계획', '도입', ...",2
3,19,매출을 올리는 가지 판매촉진 요소 이벤트를 진행하면 구매전환율이 올라간다 마트에 ...,"['고객', '휘트', '체험', '도입', '이유', '최대한', '이때', '수...",2
4,38,다른 자사몰 담당자들은 리뷰를 어떻게 활용하고 있을까 지난 일부터 일까지 크리마는 ...,"['고객', '지피지기', '영향', '응답자', '입력', '소셜', '영향력',...",2
...,...,...,...,...
21645,492838,애드드림즈 회사 스마트스토어 마케팅 문의하기 전인데요 애드드림즈 회사에 스마트스토어...,"['요청', '추천', '애드', '노출', '드리', '스토어', '스마트', '...",1
21646,492908,스마트스토어상위노출 어떤 알고리즘으로 움직이나요 안녕하세요 이번에 저희 매장에서 판...,"['음식', '스마트', '조리', '상위', '키워드', '매장', '준비', '...",1
21647,493215,네이버공식대행사를 통해 스토어팜 광고하고싶은데요 부정클릭무료로 잡아주는 대행사가 있...,"['부정', '광고', '결정', '무료', '엄청', '네이버', '대행사', '...",1
21648,493354,내공 스마트스토어 리뷰하는곳 추천 스마트스토어 리뷰를 광고비주고 쓰고싶습니다 추천하...,"['스마트', '냠냠', '대략', '리뷰', '내공', '감사', '추천', '광...",1


In [104]:
okt = Okt()
def preprocess_okt(text):
    #text = spacing(text) #띄어쓰기 보정 위에서 했으면 필요없습니다
    pos_words = okt.pos(text, stem=True)
    words = [word for word, tag in pos_words if tag in ['Noun', 'Adjective', 'Verb'] ]
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','아니다',
                 '이렇다','저렇다','도','다','돼다','최고','되다','를','같다',
                 '되었다','오다','가다','으로','자','에','와','한','하다','것',
                 '수','있다','않다','그','이다','이었다', '안','억','이건','정도',
                 '개']
    stopped_words = [w for w in words if not w in stopwords]
    return ', '.join(stopped_words)

In [105]:
tqdm.pandas()

df['review_token']=df['review'].progress_apply(preprocess_okt)
df['review_token']= [ _ for _ in df.review_token]

#빈 값 확인
# empty_index = [index for index, sentence in zip(df.index,df['review_token']) if len(sentence) < 1]
# print(empty_index)
df

100%|██████████| 14231/14231 [12:31<00:00, 18.93it/s]


,Unnamed: 0,review,Mecab_token,cluster,review_token
0,76,손실을 줄이는 슬기로운 폐업 중소 연구원과 통계청에 따르면 현재 고용원이 있는 자영...,"['가격', '고용원', '세무서', '내용증명', '규정', '재고', '가산세'...",3,"손실, 줄이다, 슬기롭다, 폐업, 중소, 연구원, 통계청, 따르다, 현재, 고용, ..."
1,206,설명 들었으니 팁을 달라고요 호텔에서 먹은 조식이 기대 이상으로 맛있었다 멕시코는 ...,"['알아내', '가격', '과정', '요청', '나이', '쏟아지', '거절', '...",3,"설명, 들다, 팁, 달다, 호텔, 먹다, 조식, 기대, 이상, 맛있다, 멕시코, 원..."
2,263,선환불 보상 안해주려 각종 트집잡는 ㅋㅍ 추가상황 아 이제 정말 상품 삭제...,"['계좌', '고객', '요청', '입력', '비대', '송금', '협조', '악의...",3,"선, 환불, 보상, 해주다, 각종, 트집, 잡다, 추가, 상황, 이제, 정말, 상품..."
3,265,정말 황당합니다 여성의류 쇼핑몰 시작한지 년이 다되가네요 쇼핑몰 시작하고 쭉 함께한...,"['픽업', '요청', '감사', '암말', '당황', '미송', '최대한', '택...",3,"정말, 황당하다, 여성, 의류, 쇼핑몰, 시작, 한지, 년, 쇼핑몰, 시작, 쭉, ..."
4,270,광고 대행업체에 스토어 주매니저 권한 줘도 되나요 네이버 스마트 스토어 광고 대행을...,"['하루', '키워드', '대행', '시스템', '연동', '스토어', '매출액',...",3,"광고, 대행, 업체, 스토어, 매니저, 권한, 주다, 네이버, 스마트, 스토어, 광..."
...,...,...,...,...,...
14226,495336,핸드폰 대리점 신고하기 핸드폰대리점신고하고싶은데요 가족결합 및 인터넷결합으로 했구요...,"['인터넷', '캡쳐', '스트레스', '아직', '노트', '태도', '책임', ...",3,"핸드폰, 대리점, 신고, 핸드폰, 대리점, 신고, 싶다, 가족, 결합, 및, 인터넷..."
14227,495373,세탁소 손해배상청구 안녕하세요 이번에 세탁소아줌마랑 법적싸움을 해야할거같아 조언을 ...,"['하루', '이건', '선임', '안팎', '대신', '거짓말', '금액', '기...",3,"세탁소, 손해배상, 청구, 안녕하다, 이번, 세탁소, 아줌마, 법적, 싸움, 조언,..."
14228,495940,핸드폰 대리점에서 사기당한것 같아요 집 근처에서 폰을 샀어요 대리점 이라고 해야하나...,"['가격', '수정', '말투', '테이프', '근처', '지우', '부모', '변...",3,"핸드폰, 대리점, 사기, 당하다, 집, 근처, 폰, 사다, 대리점, 뭐, 플러스, ..."
14229,496436,세탁소 세탁물 배달 안녕하세요 세탁물 배송에 관련해 여쭤보려 합니다 세탁물을 맡기고...,"['문제', '하루', '포함', '태도', '기대', '도저히', '아시나', '...",3,"세탁소, 세탁물, 배달, 안녕하다, 세탁물, 배송, 관련, 여쭈다, 보다, 세탁물,..."


In [106]:
df.to_csv('./co-occurrence matrix/KB_SNA_0kt_cluster6_archive.csv', encoding='utf-8-sig', index=False)

# 토큰화된 단어를 리스트로 만들기

In [107]:
total_token=df['review_token'].tolist()
total_token

['손실, 줄이다, 슬기롭다, 폐업, 중소, 연구원, 통계청, 따르다, 현재, 고용, 자영, 업자, 만, 명, 지난해, 달, 보다, 만, 명, 약, 감소, 년도, 폐업, 률, 증가, 가뜩이나, 어렵다, 영업, 환경, 코로나, 겹, 치다, 더, 버티다, 못, 폐업, 자영, 업자, 년도, 많다, 음, 의미, 또한, 폐업, 자영, 업자, 애로, 사항, 조사, 상공, 시장, 진흥, 공단, 따르다, 권리금, 회수, 시설, 집기, 처분, 어렵다, 매출, 감소, 운영, 불가능하다, 폐업, 경우, 개업, 시, 지불, 권리금, 보전, 받다, 어렵다, 경우, 많다, 시설, 집기, 헐값, 처분, 등, 손실, 불가피하다, 그동안, 누적, 적자, 생각, 손해, 폐업, 시, 일정, 부분, 손해, 불가피하다, 이러하다, 손해, 꼼꼼하다, 따지다, 보고, 준비, 줄이다, 폐업, 결정, 먼저, 임대, 계약, 만료, 개월, 전, 재계약, 의사, 없다, 음, 통보, 기간, 경과, 통보, 묵시, 계약, 연장, 인정, 되어다, 통보, 일로, 개월, 지나다, 계약해지, 효력, 발생, 폐업, 당초, 예정, 지연, 되어다, 추가, 금전, 손실, 발생, 해지, 통보, 구두, 만일, 경우, 대비, 내용증명, 문자메시지, 등, 근거, 남, 수단, 활용, 좋다, 임대차, 계약, 기간, 남아, 폐업, 상황, 라면, 임대, 불가피하다, 사정, 충분하다, 설명, 이해, 협조, 구, 필요하다, 만일, 임대, 협조, 주지, 경우, 남아, 계약, 기간, 세, 주다, 폐업, 그렇다, 상황, 우려, 임대, 개월치, 세, 미리, 지급, 설득, 이익, 가게, 부동산, 중개업, 소, 내놓다, 때, 시간, 촉박하다, 권리금, 협상, 불리하다, 여유, 시간, 두다, 믿다, 만, 업소, 내놓다, 좋다, 여러, 부동산, 내놓다, 주위, 좋다, 소문, 날, 권리금, 깎다, 일, 때문, 또, 가게, 빼다, 규정, 중개, 수수료, 더, 많다, 금액, 주다, 제안, 관심, 갖다, 필요하다, 권리금, 가게, 관리, 상태, 따르다, 가감, 장사,

# 동시출현빈도 매트릭스 만들기

In [108]:
# 동시출현빈도 매트릭스
cv=CountVectorizer()

X = cv.fit_transform(total_token)
Xc = (X.T * X)
Xc.setdiag(0) #대각선 원소를 0으로
Xc=Xc.todense() # 밀집행렬로 변형 (matrix형으로)
Xc

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [109]:
names = cv.get_feature_names_out()

#Xc.toarray() .todense와 같지만 array형태로 만들어줘서 dataframe에 넣음
matrix = pd.DataFrame(data =Xc, columns = names, index = names)
matrix.index

Index(['가가', '가가호호', '가각', '가감', '가걱', '가건물', '가게', '가겐', '가겔', '가격',
       ...
       '힘쓰다', '힘없다', '힘주다', '힘줄', '힘즘주세', '힘차다', '힙내세', '힙니', '힙듭니', '힙합'],
      dtype='object', length=36028)

In [110]:
matrix

,가가,가가호호,가각,가감,가걱,가건물,가게,가겐,가겔,가격,...,힘쓰다,힘없다,힘주다,힘줄,힘즘주세,힘차다,힙내세,힙니,힙듭니,힙합
가가,0,0,0,0,0,0,43,0,0,920,...,1,1,0,0,0,2,0,0,0,0
가가호호,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
가각,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
가감,0,0,0,0,0,0,18,0,0,12,...,0,0,0,0,0,0,0,0,0,0
가걱,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
힘차다,2,0,0,0,0,0,33,0,0,39,...,0,0,0,0,0,0,0,0,0,0
힙내세,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
힙니,0,0,0,0,0,0,0,0,0,9,...,0,0,0,0,0,0,0,0,0,0
힙듭니,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
matrix.to_csv('./co-occurrence matrix/co_occurrence matrix_cluster6.csv', encoding = 'utf-8-sig', sep = ',')

# 만약 평균 기준으로 cutoff 한다면!
- 노드 빈도가 평균 이상일 경우만 살려주고 싶다면 사용하세요
- 데이터를 가볍게 만들길 원한다면,,

[reference]

하지만 빈도수 매트릭스의 경우 모든 키워드에 연결관계가 나타나 복잡할 가능성이 존재하기 때문에 매트릭스의 평균값을 기준(cut-off value)으로 평균보다 클 경우 1을, 작을 경우 0을 반환하여(slicing)새로운 바이너리 매트릭스(binary matrix)를 생성하였다(정풀잎 외, 2019; John Scott, 2012).

In [112]:
mean_val = np.mean(Xc[Xc!=0]) # Xc의 평균값 계산
Xc[Xc < int(mean_val)] = 0 # 평균 이하인 값을 0으로 바꾸기

In [113]:
# Xc를 numpy array로 변환
Xc = np.array(Xc)

# 각 행과 열의 합 구하기
row_sum = np.sum(Xc, axis=1)
col_sum = np.sum(Xc, axis=0)

# 합이 0인 인덱스 찾기
zero_row_idx = np.where(row_sum == 0)[0]
zero_col_idx = np.where(col_sum == 0)[0]

# 해당 인덱스 삭제
Xc = np.delete(Xc, zero_row_idx, axis=0)
Xc = np.delete(Xc, zero_col_idx, axis=1)

# 결과 출력
print(Xc)
Xc.shape

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


(20249, 20249)

In [114]:
names = cv.get_feature_names_out()
Name = [val for i, val in enumerate(names) if i not in zero_row_idx]
matrix = pd.DataFrame(data =Xc, columns = Name, index = Name)
matrix.index

Index(['가가', '가감', '가건물', '가게', '가겐', '가격', '가격딜', '가격택', '가격파괴', '가격표',
       ...
       '힘들엇죠', '힘빠지다', '힘뺼시간', '힘쓰다', '힘없다', '힘주다', '힘즘주세', '힘차다', '힙니',
       '힙듭니'],
      dtype='object', length=20249)

In [115]:
matrix

,가가,가감,가건물,가게,가겐,가격,가격딜,가격택,가격파괴,가격표,...,힘들엇죠,힘빠지다,힘뺼시간,힘쓰다,힘없다,힘주다,힘즘주세,힘차다,힙니,힙듭니
가가,0,0,0,43,0,920,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
가감,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
가건물,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
가게,43,0,0,0,0,1763,0,0,0,0,...,0,0,0,0,0,0,0,33,0,0
가겐,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
힘주다,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
힘즘주세,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
힘차다,0,0,0,33,0,39,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
힙니,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
matrix.to_csv('./co-occurrence matrix/co_occurrence_matrix_cutoff_cluster6.csv', encoding = 'utf-8-sig', sep = ',')